## CNN character classifier evaluation

In [2]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Dropout
from keras.layers import Dense, Flatten
from keras.utils import np_utils
import scipy.io as sio
import numpy as np
import cv2
import h5py

In [3]:
# Load the dataset
mat = sio.loadmat("../matlab/emnist-bymerge.mat")
mapping = {kv[0]:kv[1:][0] for kv in mat['dataset'][0][0][2]}

# Extract the features and labels
X = np.array(mat['dataset'][0][0][0][0][0][0], 'int16')
y = np.array(mat['dataset'][0][0][0][0][0][1], 'int')

# ensure correct shape
X = X.reshape(X.shape[0], 28, 28, 1)

# one hot encode y
y = np_utils.to_categorical(y, 47)

In [6]:
from keras.models import load_model
from sklearn.model_selection import KFold

print('loading model...')
model = load_model("../classifiers/cnn-bymerge-E5.h5")

print('splitting data...')
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)
cvscores = []
histories = []
for train, test in kfold.split(X):
    print("starting fold...")
    # Fit the model
    histories.append(model.fit(X[train], y[train], epochs=10, verbose=1))
    print('done with fold...')
    # evaluate the model
    scores = model.evaluate(X[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

loading model...
splitting data...
starting fold...
Epoch 1/10
  4544/628138 [..............................] - ETA: 13:48 - loss: 15.2525 - acc: 0.0537

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(1, len(cvscores)+1), cvscores)
plt.xlabel('Fold Number')
plt.ylabel('Accuracy')
plt.show()

# Accuracy Curves
for hist in histories:
    plt.figure(figsize=[8,6])
    plt.plot(hist.history['acc'],'r',linewidth=3.0)
    plt.legend(['Training Accuracy'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Accuracy',fontsize=16)
    plt.title('Accuracy Curves',fontsize=16)
    plt.show()